In [1]:
import pandas as pd
from nba_api.stats.endpoints import leaguegamefinder
from joblib import dump

In [ ]:
from nba_api.stats.endpoints import LeagueDashTeamStats
from nba_api.stats.endpoints import LeagueGameLog
from nba_api.stats.static import teams

team_name_to_id = {}
for t in teams.get_teams():
    team_name_to_id[t['abbreviation']] = t['id']

df_team = (
    LeagueDashTeamStats (
        season= '2024-25',
        season_type_all_star='Regular Season'
    )
    .get_data_frames()[0]
)

features = ['TEAM_ID', 'W_PCT', 'PTS_RANK', 'AST_RANK', 'REB_RANK']
season_team_prev_features = df_team[features]

In [38]:
df_games = (
    LeagueGameLog(
        season = '2024-25',
        season_type_all_star='Playoffs'
    ).get_data_frames()[0]
)

df_games['WIN'] = (df_games['WL'] == 'W').astype(int)

df_games_with_season_stats = df_games.merge (
    season_team_prev_features,
    on = 'TEAM_ID',
    how = 'left'
)


new_col = []
home_col = []
for line in df_games_with_season_stats['MATCHUP']:
    team, vs, opp = line.split()
    if vs != '@':
        home_col.append(1)
    else:
        home_col.append(0)
    new_col.append(team_name_to_id[opp])

df_games_with_season_stats['OPP_ID'] = new_col
df_games_with_season_stats['is_home'] = home_col


def elo_win_prob(elo_a, elo_b):
    pt1 = 1 + pow(10, (elo_b - elo_a)/400)
    return 1 / pt1

# elo rankings for the teams as the season goes on

team_elos = {}
for id in df_games_with_season_stats['TEAM_ID'].unique():
    team_elos[id] = 1500


K = 20 # learning rate, essentially how dramatically the ranking moves game by game
# whether the team loses or wins

pre_elo = []
pre_opp_elo = []
df_games_with_season_stats = df_games_with_season_stats.sort_values(['GAME_DATE','TEAM_ID']).reset_index(drop=True)

for _, r in df_games_with_season_stats.iterrows():
    team_id = r['TEAM_ID']
    opp_id = r['OPP_ID']

   
    e_t = team_elos[team_id]
    e_o = team_elos[opp_id]
    
    pre_elo.append(e_t)
    pre_opp_elo.append(e_o)

    expected_prob = elo_win_prob(e_t, e_o)
    s_a = r['WIN']


    team_elos[team_id] = e_t + K * (s_a - expected_prob)
    team_elos[opp_id] = e_o + K * ((1 - s_a) - (1 - expected_prob))

dump(team_elos, 'team_elos.joblib')


['team_elos.joblib']

In [41]:
# momentum
df_games_with_season_stats['momentum_5'] = (
    df_games_with_season_stats.groupby('TEAM_ID')['WIN']
        .rolling(window = 5, min_periods=1)
        .mean()
        .shift(1) # prevent leakage of knowing the Game N's result
        .reset_index(level=0, drop=True)
)

df_games_with_season_stats['momentum_5'] = df_games_with_season_stats['momentum_5'].fillna(0)

In [ ]:
latest_momentum = df_games_with_season_stats.groupby('TEAM_ID')['momentum_5'].last()
dump(latest_momentum, 'last_mom5.joblib')